<a href="https://colab.research.google.com/github/Neulvo/TIL/blob/master/B_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [187]:
import pandas as pd

In [188]:
!git clone https://github.com/entelecheia/eKoNLPy.git
!pip install ./eKoNLPy

fatal: destination path 'eKoNLPy' already exists and is not an empty directory.
Processing ./eKoNLPy
  Created wheel for eKoNLPy: filename=eKoNLPy-0.5.30-cp36-none-any.whl size=14437140 sha256=94793e71673594a657341228ebb7e6af59dc677760340e1efde4f7c02756c634
  Stored in directory: /tmp/pip-ephem-wheel-cache-bv4mm7f5/wheels/55/6a/be/07f5aa04d32b20f90e33fc67d4939df5ead326f04db4d4381c
Successfully built eKoNLPy
  Found existing installation: eKoNLPy 0.5.30
    Uninstalling eKoNLPy-0.5.30:
      Successfully uninstalled eKoNLPy-0.5.30


In [189]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 113 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded

In [190]:
import re
import requests # 웹 상의 데이터를 가져올 때
from bs4 import BeautifulSoup #가져온 데이터에서 필요한 정보를 파싱할 때
from tqdm.notebook import tqdm


In [191]:
# for page in range(1, 11):
# for page in range(1, 101, 10):
# for page in range(10):
no_page =126

report_link = []
report_yymm = []
for page in range(no_page):
  url = 'https://finance.naver.com/research/debenture_list.nhn'
  query={
    'page' : page+1
  }
  resp=requests.get(url,query)
  soup= BeautifulSoup(resp.content, 'lxml')
  a_tag=soup.select('tr td.file a')
  b_tag=soup.select('tr td.date')
  c_tag=[]
  for idx, b in enumerate(b_tag):
    if idx%2==0:
      c_tag.append(b)

  for a, c in zip(a_tag,c_tag):
    report_link.append(a['href'])
    report_yymm.append('20'+c.text)

len(report_link)


30

In [193]:
from io import open

In [194]:
import requests
from bs4 import BeautifulSoup
# file_res = requests.get("https://ssl.pstatic.net/imgstock/upload/research/debenture/1595205128882.pdf") # 여기에 url을 for문으로 돌릴거고
# with open('{}.pdf', 'wb') as f: # 여기에 pdf 이름을 정하면 {위에 있는 코드에서 {}.pdf에 넣어주면 돼}
#   f.write(file_res.content)

In [195]:
!pip install tika

In [196]:
from tika import parser

In [197]:
full_pdf=[]
for link, yymm in tqdm(zip(report_link, report_yymm)):
  file_res = requests.get(link)
  with open('{}.pdf', 'wb') as f:
    f.write(file_res.content)
  # fp = open('{}.pdf', 'rb') # 여기에 저장된 url에 대한 response.content를 넣어줘야 함
  try:
    raw= parser.from_file('{}.pdf')
  except:
    pass
  v=raw['content']
  v=re.sub('[\n]', '', v)
  v=re.sub('[A-z]','',v)
  v=re.sub('<[^>]*>','',v)
  v=re.sub('\(+\)','',v)
  full_pdf.append([yymm,v])


In [198]:
F=pd.DataFrame(full_pdf, columns=['date','contents'])

In [199]:
F

,date,contents
0,2010.10.14,14 2010 2010년 10월 금융통화위원회 계속 직진 또는 1 시 방...
1,2010.10.14,"- 20101014 코멘트푸르덴셜. 3개월 연속 기준금리 동결, ..."
2,2010.10.14,- 101014 본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정...
3,2010.10.13,- 101012. 1 자료제공일: 2010년 10월 13일 ...
4,2010.10.13,- 101013 본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정...
5,2010.10.12,- 101012 본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정...
6,2010.10.12,- -가격부담으로 약세 기록. & 민동원(6309-4691/@...
7,2010.10.11,1 미국 고용 추이 미국 주가 및 ...
8,2010.10.11,- 101011. 1 금통위 불확...
9,2010.10.11,- 01011 본 조사분석자료는 당사 리서치센터에서 신뢰할 만한 자료 및 정보...


In [200]:
F['contents']=F['contents'].apply(lambda x : x.split('.'))

In [201]:
call_lb=pd.read_csv('/content/drive/My Drive/NLP/실습 코드/BOK/Callrate_labeling.csv')

In [202]:
F=F.merge(call_lb,on='date')

In [203]:
from ekonlpy.sentiment import MPCK
pck= MPCK()

In [204]:
repeat_per=[]
for idx, ff in enumerate(F['contents']):
  token_only=[]
  for jdx, fff in enumerate(ff):
    tokk=[]
    tokens=pck.tokenize(fff)
    if tokens !=[]:
      token_only.append(tokens)
      repeat_per.append([len(list(set(tokens)))/len(tokens),tokens])
  F['contents'][idx]=token_only

In [205]:
F['contents']

0     [[계속/MAG, 직진/NNG, 방향/NNG, 우회전/NNG, 갭/NNG, 없/VA...
1     [[푸르/VA], [연속/NNG, 금리/NNG, 동결/NNG, 환율/NNG, 여건/...
2     [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
3     [[자료/NNG, 제공/NNG, 전망/NNG, 인상/NNG, 불확실성/NNG, 지속...
4     [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
5     [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
6     [[가격/NNG, 부담/NNG, 약세/NNG, 기록/NNG], [가격/NNG, 부담...
7     [[고용/NNG, 추이/NNG, 주가/NNG, 금리/NNG, 추이/NNG, 민간/N...
8     [[불확실성/NNG, 해소/NNG, 어렵/VA, 채권/NNG, 금리/NNG, 금리/...
9     [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
10    [[강하/VA, 견제구/NNG, 아직/MAG, 주자/NNG, 민첩/NNG, 실세/N...
11    [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
12    [[중/NNG, 유동성/NNG, 감소/NNG, 가격/NNG, 부담/NNG, 시장/N...
13    [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
14    [[원달러/NNG, 강세/NNG, 추가/NNG, 양적완화/NNG, 기대/NNG, 시...
15    [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
16    [[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...
17    [[원화/NNG, 강세/NNG, 금리/NNG, 인상/NNG], [원화/NNG

In [206]:
F['contents'][12]

[['중/NNG',
  '유동성/NNG',
  '감소/NNG',
  '가격/NNG',
  '부담/NNG',
  '시장/NNG',
  '국고채/NNG',
  '물/NNG',
  '국고채/NNG',
  '물/NNG',
  '보합/NNG',
  '상승/NNG',
  '하/VV'],
 ['기록/NNG', '하/XSV', '통안채/NNG', '물/NNG', '하락/NNG', '하/VV'],
 ['기록/NNG',
  '하/XSV',
  '약보합/NNG',
  '보/VV',
  '채/NNG',
  '강세/NNG',
  '보/VV',
  '강세/NNG',
  '출발/NNG',
  '하/XSV',
  '결국/MAG',
  '가격/NNG',
  '부담/NNG',
  '느끼/VV',
  '의하/VV',
  '장후반/NNG',
  '약세/NNG',
  '반전/NNG',
  '하/VV',
  '보/VV',
  '장중/NNG',
  '발표/NNG',
  '되/VV',
  '통화/NNG',
  '증가율/NNG',
  '평잔/NNG',
  '기준/NNG'],
 ['그치/VV',
  '시중/NNG',
  '유동성/NNG',
  '감소/NNG',
  '따르/VV',
  '채권/NNG',
  '매수/NNG',
  '감소/NNG',
  '우려/NNG',
  '영향/NNG',
  '끼친/VV',
  '보/VV'],
 ['소매/NNG', '판매가격/NNG'],
 ['기록/NNG'],
 ['증가/NNG',
  '하/XSV',
  '달러/NNG',
  '강세/NNG',
  '보/VV',
  '경기/NNG',
  '변동성/NNG',
  '증가/NNG',
  '우려/NNG',
  '감소/NNG',
  '하/XSV'],
 ['시장/NNG',
  '여전히/MAG',
  '가격/NNG',
  '부담/NNG',
  '느끼/VV',
  '약세/NNG',
  '보/VV',
  '가능성/NNG',
  '크/VA'],
 ['다시/MAG', '약보합장/NNG', '연출/NNG', '크레딧/NNG', '보합/NNG'],
 

In [207]:
F

,date,contents,Labeling
0,2010.10.14,"[[계속/MAG, 직진/NNG, 방향/NNG, 우회전/NNG, 갭/NNG, 없/VA...",down
1,2010.10.14,"[[푸르/VA], [연속/NNG, 금리/NNG, 동결/NNG, 환율/NNG, 여건/...",down
2,2010.10.14,"[[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...",down
3,2010.10.13,"[[자료/NNG, 제공/NNG, 전망/NNG, 인상/NNG, 불확실성/NNG, 지속...",up
4,2010.10.13,"[[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...",up
5,2010.10.12,"[[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...",-
6,2010.10.12,"[[가격/NNG, 부담/NNG, 약세/NNG, 기록/NNG], [가격/NNG, 부담...",-
7,2010.10.11,"[[고용/NNG, 추이/NNG, 주가/NNG, 금리/NNG, 추이/NNG, 민간/N...",-
8,2010.10.11,"[[불확실성/NNG, 해소/NNG, 어렵/VA, 채권/NNG, 금리/NNG, 금리/...",-
9,2010.10.11,"[[조사/NNG, 분석/NNG, 자료/NNG, 신뢰/NNG, 하/VV, 자료/NNG...",-
